# 0. 라이브러리 불러오기

In [1]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
from transformers import TFBertForSequenceClassification
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import pandas as pd
import numpy as np

2022-08-03 13:54:36.090607: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2022-08-03 13:54:38.339898: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-08-03 13:54:38.340085: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-03 13:54:38.342694: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:00:1e.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.56GiB deviceMemoryBandwidth: 298.08GiB/s
2022-08-03 13:54:38.342721: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2022-08-03 13:54:38.345418: I tensorflow/stream_executor/platform/default

# 1. 학습시킬 데이터 불러오기

In [2]:
data = pd.read_csv('최종학습데이터_전처리완_EDA완.csv')
data.drop(labels='Unnamed: 0', axis=1, inplace=True)
data.columns = ['desc', 'label']
data

,desc,label
0,새상품입니다\n찐 아님,0
1,s급 se2 화이트 64g 팝니다\n사자마자 범퍼끼우고 다녀서 기스하나 없습니다\n...,1
2,용량 64\n \n개통일 18813\n액정 깨끗하고 테두리 찍힘만 살짝있는\n기...,3
3,배터리 88퍼 \n수리,2
4,\n갤럭시노트10 아우라글로우색상 256G 판매합니다\n외관상태 아래사진과같이 매우...,1
...,...,...
23581,그을려서 있구요\n사용하는데 문제는 없지만\n고데기에 찍힘 메인보드이상으로\n와이파...,4
23582,갤럭시 노트10 5g 256기가 팝니다 \n유심 3사가능하구요 그냥 유심 끼고 바로...,4
23583,잔상 딸애주려고 찍힘없고 충전단자옆 기스만 있어요 기기빠르고 버벅거림없고 있으며 배...,4
23584,외관은 폰으로 구입해서 처분합니다\n뒷판에 바코드 붙어있음 용량확인\n기능 및 더큰...,4


In [3]:
data.dropna(how='any', inplace=True)

# 2. 학습, 검증 데이터 분리

In [4]:
train_texts = data['desc'].to_list()
train_labels = data['label'].to_list()

In [5]:
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.2, random_state=0)

# 3. 토큰화

In [6]:
# Load Tokenizer
tokenizer = BertTokenizer.from_pretrained('kykim/bert-kor-base')

# Tokenizing
train_encodings = tokenizer(train_texts, return_tensors='pt', truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts, return_tensors='pt', truncation=True, padding=True, max_length=128)

# 4. 학습을 위한 데이터셋 만들기

In [7]:
# trainset-set
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))

# validation-set
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
))

# 5. 모델 생성

In [8]:
LMKor_model = TFBertForSequenceClassification.from_pretrained('kykim/bert-kor-base', num_labels=6, from_pt=True)

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
LMKor_model.compile(optimizer=optimizer, loss=LMKor_model.compute_loss, metrics=['accuracy'])

2022-08-03 13:55:06.774481: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2022-08-03 13:55:07.279805: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification

# 6. 모델 학습

In [9]:
callback_earlystop = EarlyStopping(
    monitor="val_loss", 
    min_delta=0.001, # the threshold that triggers the termination (acc should at least improve 0.001)
    patience=3)

callback_modelcheckpoint = ModelCheckpoint(
    filepath='tf_model.h5',
    monitor="val_loss",
    save_best_only=True,
    mode='min',
    save_weights_only=True
)

LMKor_model.fit(
    train_dataset.shuffle(1000).batch(8), epochs=15, batch_size=8,
    validation_data=val_dataset.shuffle(1000).batch(8),
    callbacks = [callback_earlystop, callback_modelcheckpoint]
)

Epoch 1/15
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


/home/ubuntu/anaconda3/envs/python3/lib/python3.8/site-packages/tensorflow/python/autograph/impl/api.py:376: FutureWarning: The old compute_loss method is deprecated as it conflicts with the Keras compute_loss method added in TF 2.8. If you want the original HF compute_loss, please call hf_compute_loss() instead. From TF versions >= 2.8, or Transformers versions >= 5, calling compute_loss() will get the Keras method instead.
  return py_builtins.overload_of(f)(*args)
2022-08-03 13:55:19.107499: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2022-08-03 13:55:19.217169: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2499990000 Hz


2359/2359 [==============================] - 580s 239ms/step - loss: 0.6865 - accuracy: 0.7674 - val_loss: 0.5299 - val_accuracy: 0.8336
Epoch 2/15
2359/2359 [==============================] - 568s 241ms/step - loss: 0.4181 - accuracy: 0.8663 - val_loss: 0.4336 - val_accuracy: 0.8671
Epoch 3/15
2359/2359 [==============================] - 550s 233ms/step - loss: 0.3020 - accuracy: 0.9071 - val_loss: 0.4418 - val_accuracy: 0.8796
Epoch 4/15
2359/2359 [==============================] - 541s 229ms/step - loss: 0.2448 - accuracy: 0.9267 - val_loss: 0.3950 - val_accuracy: 0.8875
Epoch 5/15
2359/2359 [==============================] - 544s 231ms/step - loss: 0.2010 - accuracy: 0.9439 - val_loss: 0.3836 - val_accuracy: 0.8951
Epoch 6/15
2359/2359 [==============================] - 549s 233ms/step - loss: 0.1793 - accuracy: 0.9473 - val_loss: 0.4054 - val_accuracy: 0.9021
Epoch 7/15
2359/2359 [==============================] - 557s 236ms/step - loss: 0.1505 - accuracy: 0.9557 - val_loss: 0.423

# 7. 모델 저장

In [10]:
import os

MODEL_NAME = 'fine-tuned-kykim-bert-base'
MODEL_SAVE_PATH = os.path.join("end_model", MODEL_NAME) # change this to your preferred location

if os.path.exists(MODEL_SAVE_PATH):
    print(f"{MODEL_SAVE_PATH} -- Folder already exists \n")
else:
    os.makedirs(MODEL_SAVE_PATH, exist_ok=True)
    print(f"{MODEL_SAVE_PATH} -- Folder create complete \n")

# save tokenizer, model
LMKor_model.save_pretrained(MODEL_SAVE_PATH)
tokenizer.save_pretrained(MODEL_SAVE_PATH)

end_model/fine-tuned-kykim-bert-base -- Folder create complete 



('end_model/fine-tuned-kykim-bert-base/tokenizer_config.json',
 'end_model/fine-tuned-kykim-bert-base/special_tokens_map.json',
 'end_model/fine-tuned-kykim-bert-base/vocab.txt',
 'end_model/fine-tuned-kykim-bert-base/added_tokens.json')

# 8. 테스트

In [15]:
from transformers import TextClassificationPipeline

# Load Fine-tuning model
loaded_tokenizer = BertTokenizer.from_pretrained(MODEL_SAVE_PATH)
loaded_model = TFBertForSequenceClassification.from_pretrained(MODEL_SAVE_PATH, id2label={0: 0 , 1: 1, 2: 2, 3: 3, 4: 4, 5: 5})

text_classifier = TextClassificationPipeline(
    tokenizer=loaded_tokenizer, 
    model=loaded_model, 
    framework='tf',
    return_all_scores=True
)

Some layers from the model checkpoint at end_model/fine-tuned-kykim-bert-base were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at end_model/fine-tuned-kykim-bert-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.
/home/ubuntu/anaconda3/envs/python3/l

In [16]:
testset = pd.read_excel('테스트셋.xlsx')

In [17]:
predicted_label_list = []
predicted_score_list = []

for text in testset['desc']:
    # predict
    preds_list = text_classifier(text)

    sorted_preds_list = sorted(preds_list, key=lambda x: x['score'], reverse=True)
    predicted_label_list.append(sorted_preds_list[0]['label']) # label
    predicted_score_list.append(sorted_preds_list[1]['score']) # score
testset['pred'] = predicted_label_list
testset['score'] = predicted_score_list

In [19]:
from sklearn.metrics import classification_report

print(classification_report(y_true=testset['label'], y_pred=testset['pred']))

              precision    recall  f1-score   support

           0       0.92      1.00      0.96        11
           1       1.00      0.69      0.82        13
           2       0.73      0.65      0.69        17
           3       0.70      1.00      0.82        14
           4       0.81      0.81      0.81        16
           5       0.88      0.83      0.86        18

    accuracy                           0.82        89
   macro avg       0.84      0.83      0.83        89
weighted avg       0.83      0.82      0.82        89

